# **Instructions**:

### 1 - Open your game that you want to perform detections
### 2 - In the game window, get the name of it's title bar 
### 3 - Update the variable "window_name" with the game title bar name
### 4 - Run all cells to start detecting objects using your trained model

In [9]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os
import random

In [10]:
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img) 
            
        return img

    def generate_image_dataset(self):
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)
    
    def get_window_size(self):
        return (self.w, self.h)

In [11]:
class ImageProcessor:
    W = 0
    H = 0
    net = None
    ln = None
    classes = {}
    colors = []

    def __init__(self, img_size, cfg_file, weights_file):
        np.random.seed(42)
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i-1] for i in self.net.getUnconnectedOutLayers()]
        self.W = img_size[0]
        self.H = img_size[1]
        
        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()
        
        # If you plan to utilize more than six classes, please include additional colors in this list.
        self.colors = [
            (0, 0, 255), 
            (0, 255, 0), 
            (255, 0, 0), 
            (255, 255, 0), 
            (255, 0, 255), 
            (0, 255, 255)
        ]
        

    def proccess_image(self, img):

        blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)
        
        coordinates = self.get_coordinates(outputs, 0.5)

        self.draw_identified_objects(img, coordinates)

        return coordinates

    def get_coordinates(self, outputs, conf):

        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w//2), int(y - h//2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})
        return coordinates

    def draw_identified_objects(self, img, coordinates):
        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']
            
            color = self.colors[classID]
            
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
#        cv.imshow('window',  img)


In [ ]:
# Run this cell to initiate detections using the trained model.
import keyboard
import pyautogui

window_name = "Harbi2 Global | International Official\xa0Server - Doyyumhwan"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_final.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)
selam = 0
while(True):
    sleep(2)
    
    ss = wincap.get_screenshot()
    
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break
        
    # keyboard.press('`')
    coordinates = improc.proccess_image(ss)
    
    for coordinate in coordinates:
        print(coordinate)
    print('=====')

    first_metin = coordinates[0]
    print(first_metin)
    pyautogui.click(
        x= first_metin['x'] + (first_metin['w'] / 2), 
        y= first_metin['y'] + (first_metin['h'] / 2)
    )
    
    
    # If you have limited computer resources, consider adding a sleep delay between detections.
    sleep(2)
    selam = selam +1
    exit(1)

print('Finished.')

C:\Users\selet\AppData\Local\Temp\ipykernel_18956\2781964491.py:32: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


{'x': 1329, 'y': 1122, 'w': 41, 'h': 37, 'class': 0, 'class_name': 'metin'}
{'x': 2077, 'y': 855, 'w': 47, 'h': 41, 'class': 0, 'class_name': 'metin'}
{'x': 1467, 'y': 265, 'w': 44, 'h': 50, 'class': 0, 'class_name': 'metin'}
{'x': 2041, 'y': 265, 'w': 48, 'h': 36, 'class': 0, 'class_name': 'metin'}
{'x': 659, 'y': 379, 'w': 51, 'h': 37, 'class': 0, 'class_name': 'metin'}
{'x': 1209, 'y': 678, 'w': 33, 'h': 15, 'class': 0, 'class_name': 'metin'}
{'x': 811, 'y': 871, 'w': 44, 'h': 40, 'class': 0, 'class_name': 'metin'}
=====
{'x': 1329, 'y': 1122, 'w': 41, 'h': 37, 'class': 0, 'class_name': 'metin'}
{'x': 1329, 'y': 1122, 'w': 46, 'h': 39, 'class': 0, 'class_name': 'metin'}
{'x': 2041, 'y': 261, 'w': 46, 'h': 41, 'class': 0, 'class_name': 'metin'}
{'x': 655, 'y': 377, 'w': 53, 'h': 39, 'class': 0, 'class_name': 'metin'}
{'x': 2076, 'y': 856, 'w': 44, 'h': 43, 'class': 0, 'class_name': 'metin'}
{'x': 364, 'y': 808, 'w': 44, 'h': 37, 'class': 0, 'class_name': 'metin'}
{'x': 1468, 'y': 272

In [4]:
def enum_window_titles():
    def callback(handle, data):
        titles.append(win32gui.GetWindowText(handle))

    titles = []
    win32gui.EnumWindows(callback, None)
    return titles

window_titles = enum_window_titles()
print(window_titles)

NameError: name 'win32gui' is not defined

In [13]:
import pyautogui
import pygetwindow as gw
import time

# Define the name of the window you want to capture
window_name = "Harbi2 Global | International Official\xa0Server - Doyyumhwan"

# Function to find the window by name
def find_window_by_name(window_name):
    try:
        return gw.getWindowsWithTitle(window_name)[0]
    except IndexError:
        return None

# Function to press 'a' inside the specified window
def press_a_in_window(window):
    if window is not None:
        window.activate()
        pyautogui.press('a')

# Main loop
while True:
    # Find the window by name
    target_window = find_window_by_name(window_name)
    
    # Press 'a' if the window is found
    if target_window:
        press_a_in_window(target_window)
    
    # Add a delay to control the rate of key presses
    time.sleep(0.1) 

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

# AAAAaaaaaaaaaaaaaaaa

In [ ]:
AAAaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa